# AI Agent Framework for .NET

**Important**: If you encounter version conflicts, restart the kernel and run all cells in order.

## Package References

Load all required packages first to ensure compatibility:

In [1]:
#r "nuget: Microsoft.Extensions.AI, 9.9.0"

Installed Packages Microsoft.Extensions.AI, 9.9.0

In [2]:
// #r "nuget: OpenAI, 2.4.0"

In [3]:
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Agents.AI.OpenAI\bin\Debug\net9.0/Microsoft.Agents.AI.OpenAI.dll"

In [4]:
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Agents.AI\bin\Debug\net9.0\Microsoft.Agents.AI.dll"

In [5]:
#r "nuget: Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4"

Installed Packages Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4

In [6]:
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages DotNetEnv, 3.1.1

In [7]:
using System;
using System.ClientModel;
using System.Text.Json;
using System.Text.Json.Serialization;
using Microsoft.Extensions.AI;
using Microsoft.Agents.AI;
using OpenAI;


In [8]:
 using DotNetEnv;

In [9]:
Env.Load("../../../.env");

In [10]:
System.Environment.GetEnvironmentVariable("GITHUB_ENDPOINT");

In [11]:
var github_endpoint = Environment.GetEnvironmentVariable("GITHUB_ENDPOINT") ?? throw new InvalidOperationException("GITHUB_ENDPOINT is not set.");
var github_model_id = Environment.GetEnvironmentVariable("GITHUB_MODEL_ID") ?? "gpt-4o-mini";
var github_token = Environment.GetEnvironmentVariable("GITHUB_TOKEN") ?? throw new InvalidOperationException("GITHUB_TOKEN is not set.");

In [13]:
var openAIOptions = new OpenAIClientOptions()
{
    Endpoint= new Uri(github_endpoint)
};

In [14]:


var openAIClient = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions);

In [15]:
const string AGENT_NAME ="TravelPlanAgent";

const string AGENT_INSTRUCTIONS = @"You are an planner agent.
    Your job is to decide which agents to run based on the user's request.
    Below are the available agents specialised in different tasks:
    - FlightBooking: For booking flights and providing flight information
    - HotelBooking: For booking hotels and providing hotel information
    - CarRental: For booking cars and providing car rental information
    - ActivitiesBooking: For booking activities and providing activity information
    - DestinationInfo: For providing information about destinations
    - DefaultAgent: For handling general request";

In [16]:

    public class Plan
    {
        [JsonPropertyName("assigned_agent")]
        public string? Assigned_agent { get; set; }

        [JsonPropertyName("task_details")]
        public string? Task_details { get; set; }
    }

In [17]:

    public class TravelPlan
    {
        [JsonPropertyName("main_task")]
        public string? Main_task { get; set; }

        [JsonPropertyName("subtasks")]
        public IList<Plan> Subtasks { get; set; }

    }

In [18]:

ChatClientAgentOptions agentOptions = new(name: AGENT_NAME, instructions: AGENT_INSTRUCTIONS)
{
    ChatOptions = new()
    {
        ResponseFormat = ChatResponseFormatJson.ForJsonSchema(
            schema: AIJsonUtilities.CreateJsonSchema(typeof(TravelPlan)),
            schemaName: "TravelPlan",
            schemaDescription: "Travel Plan with main_task and subtasks")
    }
};

In [19]:
AIAgent agent = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions).GetChatClient(github_model_id).CreateAIAgent(
    agentOptions);

In [20]:

Console.WriteLine(await agent.RunAsync("Create a travel plan for a family of 4, with 2 kids, from Singapore to Melbourne"));

{"main_task":"Plan a family trip from Singapore to Melbourne","subtasks":[{"task_details":"Book round-trip flights from Singapore to Melbourne for 4 people.","assigned_agent":"FlightBooking"},{"task_details":"Find and book a family-friendly hotel in Melbourne for the duration of the stay.","assigned_agent":"HotelBooking"},{"task_details":"Arrange car rental for easy transportation around Melbourne during the trip.","assigned_agent":"CarRental"},{"task_details":"Suggest family-friendly activities and attractions in Melbourne for kids and adults.","assigned_agent":"ActivitiesBooking"},{"task_details":"Provide information about Melbourne as a destination, including best times to visit and local customs.","assigned_agent":"DestinationInfo"}]}
